In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Prepare the tokenizer
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

# Separate features and labels
training_sentences = train_data['message'].tolist()
training_labels = train_data['label'].apply(lambda x: 1 if x == 'spam' else 0).tolist()

testing_sentences = test_data['message'].tolist()
testing_labels = test_data['label'].apply(lambda x: 1 if x == 'spam' else 0).tolist()

# Tokenize
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

train_sequences = tokenizer.texts_to_sequences(training_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

train_labels = tf.convert_to_tensor(training_labels)
test_labels = tf.convert_to_tensor(testing_labels)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))


In [ ]:
def predict_message(msg):
    sequence = tokenizer.texts_to_sequences([msg])
    padded = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    prediction = model.predict(padded)[0][0]
    label = "spam" if prediction > 0.5 else "ham"
    return [float(prediction), label]
